### Tips for development vs tutorial hygiene:
---
- Keep a scratch notebook (e.g., `prtecan_devel.ipynb`) for experiments.
- Avoid `os.chdir`; use Path objects relative to repository root as in this notebook.
- When a feature stabilizes, port minimal, clear examples into the main tutorial and keep heavy testing in `tests/`.

## Setup

In [ ]:
# Magic commands for development
%load_ext autoreload
%autoreload 2

from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

from clophfit import prtecan
from clophfit.fitting.bayes import (
    fit_binding_pymc,
    fit_binding_pymc2,
    fit_binding_pymc_compare,
)
from clophfit.fitting.core import (
    fit_binding_glob,
    fit_binding_glob_recursive,
    fit_binding_glob_recursive_outlier,
    fit_binding_glob_reweighted,
    outlier2,
)
from clophfit.fitting.odr import (
    fit_binding_odr,
    fit_binding_odr_recursive,
    fit_binding_odr_recursive_outlier,
)

# Configure notebook
%matplotlib inline
plt.style.use("seaborn-v0_8")

data_root = Path("tests/Tecan")
l0_dir = data_root / "140220"
l1_dir = data_root / "L1"
l2_dir = data_root / "L2"
l4_dir = data_root / "L4"

In [ ]:
def tit(folder, bg_mth="meansd"):
    tit = prtecan.Titration.fromlistfile(folder / "list.pH.csv", is_ph=1)
    tit.load_additions(folder / "additions.pH")
    tit.load_scheme(folder / "scheme.txt")
    tit.params.bg_mth = bg_mth
    tit.params.bg_adj = True
    return tit


tit = tit(l2_dir)
tit.bg_err

## residual cov

In [ ]:
tit.result_global.compute_all()

In [ ]:
from dataclasses import dataclass

import pandas as pd

from clophfit.fitting.core import FitResult


@dataclass(frozen=True)
class ResidualPoint:
    """For single points."""

    label: str
    x: float
    resid_weighted: float  # lmfit residual = (y - model) / y_err
    resid_raw: float  # (y - model)
    i: int  # index within that label (after masking)


def residual_points(fr: FitResult) -> list[ResidualPoint]:
    if fr.result is None or fr.dataset is None:
        return []
    r = np.asarray(fr.result.residual, float)
    pts: list[ResidualPoint] = []

    start = 0
    for lbl, da in fr.dataset.items():
        n = len(da.y)  # masked length
        rw = r[start : start + n]
        rr = rw * da.y_err  # undo weighting (since rw = (y-model)/y_err)
        xs = da.x
        pts.extend(
            ResidualPoint(
                label=lbl,
                x=float(xs[i]),
                resid_weighted=float(rw[i]),
                resid_raw=float(rr[i]),
                i=i,
            )
            for i in range(n)
        )
        start += n
    if start != len(r):
        msg = f"Residual length mismatch: consumed {start}, residual has {len(r)}"
        raise ValueError(msg)
    return pts


def residual_frame(fr: FitResult) -> pd.DataFrame:
    return pd.DataFrame([p.__dict__ for p in residual_points(fr)])

In [ ]:
all_res = pd.DataFrame()
for k in tit.fit_keys:
    fr = tit.result_global[k]
    all_res = pd.concat([all_res, residual_frame(fr)])


rows = []
for well in tit.fit_keys:
    fr = tit.result_global[well]
    df = residual_frame(fr).assign(well=well)  # <-- add well id
    rows.append(df)

all_res = pd.concat(rows, ignore_index=True)
all_res["x"] = all_res["x"].round(3)  # avoid float-key drift

In [ ]:
val_col = "resid_weighted"  # or "resid_weighted"
cov_by_label: dict[str, pd.DataFrame] = {}

for lbl, g in all_res.groupby("label"):
    # rows = wells/curves, cols = x points
    M = g.pivot_table(index="well", columns="x", values=val_col, aggfunc="mean")
    # drop wells missing any x (to make a clean covariance across x points)
    M = M.dropna(axis=0, how="any")

    X = M.to_numpy(dtype=float)
    # covariance across x-points (features), so rowvar=False
    C = np.cov(X, rowvar=False, ddof=1)

    cov_by_label[lbl] = pd.DataFrame(
        C, index=M.columns.to_list(), columns=M.columns.to_list()
    )

# Example: covariance matrix for y2 (indexed by x values)
cov_y2 = cov_by_label["y1"]
print(cov_y2.round(3))
sns.heatmap(cov_y2.round(3), cmap="coolwarm")

In [ ]:
corr_by_label = {
    lbl: cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    for lbl, cov in ((k, v.to_numpy()) for k, v in cov_by_label.items())
}
corr_y2 = pd.DataFrame(
    corr_by_label["y2"],
    index=cov_by_label["y2"].index,
    columns=cov_by_label["y2"].columns,
)
corr_y2

In [ ]:
sns.heatmap(corr_y2, cmap="vlag")

In [ ]:
sns.lineplot(all_res, x="x", y="resid_weighted", hue="label")

## Residues distribution

In [ ]:
k = "G12"

fr1 = tit.results[1][k]
fr1.figure

In [ ]:
fr2 = tit.results[2][k]
print(fr1.result.redchi, fr2.result.redchi)
fr2.figure

In [ ]:
fr2.result.residual * fr2.dataset["2"].y_err / fr2.dataset["2"].y

In [ ]:
frg = tit.result_global[k]
print(frg.result.redchi)
frg.figure

In [ ]:
fr1.dataset["1"].y_err *= np.sqrt(fr1.result.redchi) / 2
fr2.dataset["2"].y_err *= np.sqrt(fr2.result.redchi) / 2

In [ ]:
fr1.dataset["1"].y_err, fr2.dataset["2"].y_err

In [ ]:
frg.dataset["y1"].y_err, frg.dataset["y2"].y_err

In [ ]:
from copy import deepcopy

In [ ]:
dsg = deepcopy(frg.dataset)

dsg["y1"].y_errc = np.ones_like(dsg["y1"].y_errc) * 44 * np.sqrt(9) / 3 * 3.1
dsg["y2"].y_errc = np.ones_like(dsg["y2"].y_errc) * 14 * np.sqrt(9) / 3 * 3

# weight_multi_ds_titration(dsg)
dsg

In [ ]:
fr = fit_binding_glob(dsg)
print(fr.result.redchi)
fr.figure

In [ ]:
fr.dataset

In [ ]:
fr.result.chisqr

In [ ]:
np.mean(np.abs(fr.result.residual[7:]))

In [ ]:
from clophfit.fitting.core import weight_da, weight_multi_ds_titration

In [ ]:
weight_da(fr1.dataset["1"], is_ph=1)

In [ ]:
weight_multi_ds_titration(fr1.dataset)

In [ ]:
fr_mcmc = tit.result_mcmc[k]

In [ ]:
fr_mcmc.figure

In [ ]:
fr_odr = tit.result_odr[k]
fr_odr.figure

In [ ]:
plt.plot(fr2.result.residual, "o")
plt.plot(frg.result.residual, "o")
plt.plot(fr_mcmc.result.residual, "s")
plt.plot(fr_odr.result.residual, "*")

In [ ]:
fr_odr.result.residual

In [ ]:
tr = tit.result_global
tr[k].result.residual

In [ ]:
def residual_df_all(tr) -> pd.DataFrame:
    rows = []
    for k in tr.fit_keys:
        ds = tr[k].dataset
        res = np.asarray(tr[k].result.residual, dtype=float)

        start = 0
        for label, da in ds.items():
            x = np.asarray(da.x, dtype=float)  # masked x used in fit
            n = x.size
            r = res[start : start + n]
            start += n
            rows += [
                {"k": k, "label": label, "x": float(xi), "residue": float(ri)}
                for xi, ri in zip(x, r, strict=True)
            ]

    return pd.DataFrame(rows)


df = residual_df_all(tr)
df["std_res"] = (df.residue - np.nanmean(df.residue)) / np.nanstd(df.residue, ddof=1)

In [ ]:
g = sns.displot(
    data=df,
    x="std_res",
    col="label",
    kind="hist",
    bins=60,
    stat="density",
    common_norm=False,
    height=4,
    aspect=1.2,
)
for ax in g.axes.flat:
    sns.kdeplot(
        data=df[df["label"] == ax.get_title().split(" = ")[-1]],
        x="std_res",
        ax=ax,
        lw=2,
    )
    ax.axvline(-2, ls="--", c="crimson", lw=1)
    ax.axvline(2, ls="--", c="crimson", lw=1)
    ax.set_xlim(-6, 6)
g.set_titles(col_template="{col_name}")
g.fig.suptitle("Standardized residuals (with ±2σ)", y=1.05)
plt.show()

In [ ]:
g = sns.catplot(
    data=df,
    x="x",
    y="std_res",
    col="label",
    kind="boxen",  # nicer than boxplot for big n
    showfliers=False,
    height=4,
    aspect=1.4,
    sharey=True,
)
for ax in g.axes.flat:
    ax.axhline(-2, ls="--", c="crimson", lw=1)
    ax.axhline(2, ls="--", c="crimson", lw=1)
    ax.set_xlabel("x")
    ax.set_ylabel("std_res")
    ax.tick_params(axis="x", rotation=45)
g.fig.suptitle("Std residuals vs x (per label)", y=1.05)
plt.show()

In [ ]:
out = (
    df
    .assign(out=np.abs(df["std_res"]) > 2.5)
    .groupby(["k", "label"], as_index=False)["out"]
    .mean()
    .rename(columns={"out": "outlier_rate"})
)
# plot top offenders per label
top = out.sort_values("outlier_rate", ascending=False).groupby("label").head(25)

plt.figure(figsize=(12, 6))
sns.barplot(data=top, y="k", x="outlier_rate", hue="label", dodge=False)
plt.xlabel("Outlier rate (|std_res| > 2)")
plt.ylabel("k (top 25 per label)")
plt.title("Worst wells by standardized-residual outlier rate")
plt.legend(title="label")
plt.tight_layout()
plt.show()

In [ ]:
# df[np.abs(df.std_res) > 2.5]

In [ ]:
df[np.abs(df.residue) > 3]

In [ ]:
df[np.abs(df.residue) > 2.5]

In [ ]:
df[df.std_res < -2.5]

In [ ]:
df[df.std_res < -2.5]

In [ ]:
tr = tit.result_global

residuals = [tr[k].result.residual.ravel() for k in tr.fit_keys]
residuals = np.concatenate(residuals)

all_res = residuals
std_res = (all_res - np.nanmean(all_res)) / np.nanstd(all_res, ddof=1)
std_res = residuals

# stats
k2, pval = stats.normaltest(std_res)
skew = stats.skew(std_res, bias=False)
kurt = stats.kurtosis(std_res, fisher=True, bias=False)

# plot
fig, ax = plt.subplots(figsize=(7, 5))
ax.hist(std_res, bins=40, density=True, color="#4c72b0", alpha=0.7)
x = np.linspace(-4, 4, 300)
# ax.plot(x, stats.norm.pdf(x, 0, 1), "r-", lw=2, label="N(0,1) PDF")
ax.set_xlabel("Standardized residual")
ax.set_ylabel("Density")
ax.set_title("Residual distribution (all fits in tit.results[2])")
props = {"boxstyle": "round", "facecolor": "white", "alpha": 0.8}
txt = f"n={len(std_res)}\n p={pval:.3g}\n skew={skew:.3f}\n kurt={kurt:.3f}"
ax.text(
    0.98,
    0.95,
    txt,
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="right",
    bbox=props,
)
ax.legend()
plt.show()

In [ ]:
sns.histplot(std_res, kde=True)

In [ ]:
stats.shapiro(std_res)

In [ ]:
stats.kstest(std_res, "norm")

In [ ]:
stats.probplot(std_res, plot=plt, rvalue=True)

In [ ]:
# Seaborn doesn't have qqplot, use scipy.stats instead
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Histogram with KDE
sns.histplot(std_res, kde=True, ax=ax1)

# Q-Q plot (using scipy)
stats.probplot(std_res, dist="norm", plot=ax2)
plt.show()
# +end_src

In [ ]:
tit.result_global["D03"].figure

In [ ]:
plt.plot(tit.result_odr["D03"].result.residual, "o")

## Discard detection

test:

- E10
- F10
- G09


In [ ]:
plt.plot([np.nanmean(tit.data[1][k] / tit.data[2][k].mean()) for k in tit.fit_keys])

print([
    (t[0], t[1])
    for t in [
        (k, np.nanmean(tit.data[1][k] / tit.data[2][k].mean())) for k in tit.fit_keys
    ]
    if t[1] > 2 or t[1] < 1
])

## Synthetic dataset

In [ ]:
from clophfit.testing.synthetic import (
    _sample_correlated_signals,
    _sample_from_real,
    make_dataset,
)

In [ ]:
from benchmarks.compare_fitting_methods import generate_synthetic_data

In [ ]:
ds = generate_synthetic_data(pKa=6.7, add_outliers=True)
g = ds.plot()

In [ ]:
ds["y1"].y_err / ds["y1"].y

In [ ]:
ds, truth = make_dataset(
    7, randomize_signals=1, error_model="realistic", rel_error={"y1": 0.100, "y2": 0.2}
)  # uniform simple realistic physics
g = ds.plot()
print(truth)
fr = outlier2(ds)
fr.figure

In [ ]:
ds = generate_synthetic_data(pKa=6.7, add_outliers=True)
fr = outlier2(ds, error_model="uniform")
fr.figure

In [ ]:
ds, thruth = make_dataset(
    7.0, 2000, 200, is_ph=True, n_labels=1, error_model="realistic"
)
fr = outlier2(ds)
# fr.figure
plt.plot(ds["y0"].x, fr.result.residual / ds["y0"].y, "o-")

In [ ]:
(
    ds["y0"].y_err / ds["y0"].y,
    fr.result.residual * fr.dataset["y0"].y_err / fr.dataset["y0"].y,
)

In [ ]:
Ks = []
sKs = []
for _i in range(99):
    ds = generate_synthetic_data(7.1, add_outliers=True)
    fr = outlier2(ds, error_model="uniform")
    Ks.append(fr.result.params["K"].value)
    sKs.append(fr.result.params["K"].stderr)
sns.histplot(Ks, kde=True)

In [ ]:
import pandas as pd

df = pd.DataFrame({"K": Ks, "K_err": sKs})
df = df[df.K_err < 0.5]

plt.errorbar(x=range(len(df.K)), y=df.K, yerr=df.K_err)

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True, width_ratios=[3, 1])

# Left panel
ax1.errorbar(
    x=range(len(df.K)),
    y=df.K,
    yerr=df.K_err,
    fmt="o",
    capsize=3,
    alpha=0.7,
    label="K ± error",
)
ax1.axhline(
    y=df.K.mean(),
    color="r",
    linestyle="--",
    alpha=0.7,
    label=f"Mean: {df.K.mean():.2f}",
)
ax1.set_xlabel("Index")
ax1.set_ylabel("K")
ax1.set_title(f"K values (n={len(df.K)})")
ax1.legend()
ax1.grid(alpha=0.3)

# Right panel with both histogram and KDE
ax2_hist = ax2
ax2_kde = ax2.twinx()

# Histogram
n, bins, patches = ax2_hist.hist(
    df.K, bins=30, orientation="horizontal", alpha=0.3, edgecolor="black", density=False
)

# KDE
kde = stats.gaussian_kde(df.K)
x_kde = np.linspace(df.K.min(), df.K.max(), 200)
y_kde = kde(x_kde)
# Scale KDE to match histogram visually
scale_factor = n.max() / y_kde.max()
ax2_kde.plot(y_kde * scale_factor, x_kde, "r-", linewidth=2, alpha=0.7, label="KDE")

ax2_hist.set_xlabel("Frequency")
ax2_kde.set_xlabel("Density (scaled)", color="r")
ax2_hist.set_title("Distribution")
ax2_hist.tick_params(axis="x")
ax2_kde.tick_params(axis="x", labelcolor="r")
ax2_hist.grid(alpha=0.3)

# Add statistics text box
stats_text = f"Mean: {df.K.mean():.3f}\nStd: {df.K.std():.3f}\nMin: {df.K.min():.3f}\nMax: {df.K.max():.3f}"
ax2_hist.text(
    0.7,
    0.95,
    stats_text,
    transform=ax2_hist.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox={"boxstyle": "round", "facecolor": "wheat", "alpha": 0.8},
)

plt.tight_layout()

In [ ]:
np.mean(Ks), np.median(Ks)

In [ ]:
rng = np.random.default_rng(None)

_sample_from_real(rng, "K")

In [ ]:
_sample_correlated_signals(rng)

In [ ]:
from functools import partial

rel_error = {"y1": 0.04, "y2": 0.01}
make_ds = partial(
    make_dataset,
    randomize_signals=True,
    rel_error=rel_error,
    min_error=1,
    low_ph_drop=False,
    x_error_large=0.0,
)

In [ ]:
from collections import defaultdict

values = defaultdict(list)

In [ ]:
# ds, truth = make_dataset(6.8, randomize_signals=True, error_model="physics", noise=.01, rel_error=rel_error, outlier_prob=.1, outlier_sigma=4)
# ds, truth = make_dataset(6.8, randomize_signals=True, rel_error=rel_error, min_error=1, low_ph_drop=True, low_ph_drop_magnitude=.25, low_ph_drop6_prob=.0, x_error_large=0.0, seed=1)
ds, truth = make_ds(6.8)
g = ds.plot()

fr = outlier2(ds, error_model="uniform")
fr.figure

In [ ]:
for _i in range(33):
    ds, truth = make_ds(7.2, min_error=0.1)

    fr = fit_binding_glob_reweighted(ds, "")
    values["reweighted"].append(fr.result.params["K"].value)
    fr = fit_binding_glob_recursive_outlier(ds)
    values["recursive_outlier"].append(fr.result.params["K"].value)
    fr = outlier2(ds)
    values["outlier"].append(fr.result.params["K"].value)
    # fr = fit_binding_pymc2(ds)
    fr = fit_binding_odr(ds)
    values["odr"].append(fr.result.params["K"].value)

for key in values:
    print(key, np.median(values[key]), np.mean(values[key]))

sns.histplot(values, kde=True)

In [ ]:
sns.stripplot(values)
sns.boxplot(values, saturation=0.01)

## Fitting

In [ ]:
k = "B04"

ds = tit._create_global_ds(k)
ds["y1"].y_err.mean(), ds["y2"].y_err.mean()
ds

In [ ]:
r1 = fit_binding_glob(ds)
r2 = fit_binding_glob(ds, robust=True)
r3 = fit_binding_glob_reweighted(ds, k, threshold=2.5)
r4 = fit_binding_glob_recursive(ds, tol=0.001, max_iterations=100)
r5 = fit_binding_glob_recursive_outlier(ds, tol=0.001, threshold=2)
r6 = outlier2(ds, k, threshold=3, plot_z_scores=True)
r7 = outlier2(ds, k, threshold=3, plot_z_scores=True, error_model="shot-noise")

r8 = fit_binding_odr(r1)
r9 = fit_binding_odr_recursive(r1, tol=0.001, max_iterations=100)
r10 = fit_binding_odr_recursive_outlier(r1, tol=0.001, threshold=3)

fr = r2
n_sd = 0.15 / fr.result.params["K"].stderr
print(n_sd)
r11 = fit_binding_pymc(fr, n_sd=max(n_sd, 1), n_xerr=0.682, ye_scaling=10)
r12 = fit_binding_pymc2(fr, n_sd=max(n_sd, 1), n_xerr=0.682)

buffer_sd = {"y1": fr.dataset["y1"].y_err.mean(), "y2": fr.dataset["y2"].y_err.mean()}
buffer_sd = {"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()}
print(buffer_sd)
trace_compare = fit_binding_pymc_compare(
    fr, buffer_sd=buffer_sd, learn_separate_y_mag=True, n_sd=max(n_sd, 1), n_xerr=0.682
)

In [ ]:
{"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()}

In [ ]:
ds["y1"].y_err, ds["y2"].y_err

In [ ]:
r7.dataset["y2"].y_err

In [ ]:
r1.result.chisqr

In [ ]:
r12.figure

In [ ]:
az.summary(trace_compare)

In [ ]:
# Combine log likelihoods for multi-output models before comparison
import warnings

import xarray as xr


def combine_log_likelihoods(idata):
    """Concatenate log likelihoods across all likelihood variables."""
    if not hasattr(idata, "log_likelihood"):
        return idata
    ll = idata.log_likelihood
    # Concatenate all likelihood variables along observation dimension
    arrays = [ll[var].rename({list(ll[var].dims)[-1]: "obs"}) for var in ll.data_vars]
    combined = xr.concat(arrays, dim="obs")
    new_ll = xr.Dataset({"combined": combined})
    return az.InferenceData(posterior=idata.posterior, log_likelihood=new_ll)


# Combine log likelihoods and compare (suppress Pareto-k warnings)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Estimated shape parameter of Pareto")
    comparison_results = az.compare({
        "single_y_mag": combine_log_likelihoods(r11.mini),
        "separate_y_mag": combine_log_likelihoods(r12.mini),
        "separate_y_mag_bg": combine_log_likelihoods(trace_compare),
    })

# The result is a pandas DataFrame.
# The best model has the lowest 'loo' or 'waic' value.
# The 'd_loo' column shows the difference from the best model.
# Note: warning=True in results indicates some Pareto-k > 0.7 (influential observations)
comparison_results

In [ ]:
ds2 = tit._create_ds(k, 2)
outlier2(ds2, error_model="shot-noise").figure